# Análisis de series temporales
Modelos Supervisados y No Supervisados.

## Casificacion Multiclase

En el presente documento intentaremos predecir cuanto tardata una entrega determinada, en un rango de 0 a 20 dias.

Para ello relizaremos una clasificacion multiclase con diferentes algoritmos, para intentar  definir cual predice de manera mas precisa el resultado esperado.

## Bibliotecas
Vamos a cargar las biblitecas necesarias para realizar el analisis.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

## Datos
Para empezr, cargamos los datos de los envios de Marzo de 2019.

In [2]:
cols = ['service',
        'sender_zipcode',
        'receiver_zipcode',
        'sender_state',
        'receiver_state',
        'shipment_type',
        'quantity',
        'status',
        'date_created',
        'date_sent',
        'date_visit',
        'target']

df = pd.read_csv('./shipments_BR_201903.csv', usecols=cols)

In [3]:
df.head(5)

,sender_state,sender_zipcode,receiver_state,receiver_zipcode,shipment_type,quantity,service,status,date_created,date_sent,date_visit,target
0,SP,3005,SP,5409,express,1,0,done,2019-03-04 00:00:00,2019-03-05 13:24:00,2019-03-07 18:01:00,2
1,SP,17052,MG,37750,standard,1,1,done,2019-03-19 00:00:00,2019-03-20 14:44:00,2019-03-27 10:21:00,5
2,SP,2033,SP,11040,express,1,0,done,2019-02-18 00:00:00,2019-02-21 15:08:00,2019-02-28 18:19:00,5
3,SP,13900,SP,18500,express,1,0,done,2019-03-09 00:00:00,2019-03-11 15:48:00,2019-03-12 13:33:00,1
4,SP,4361,RS,96810,express,1,0,done,2019-03-08 00:00:00,2019-03-12 08:19:00,2019-03-16 08:24:00,4


Como intentaremos predecir envios que tardan de 0 a 20 dias, los target mayores seran modificados a 20.

In [4]:
df['target'] = np.where(df['target'] > 20, 20, df['target'])

Para poder incluir el tipo de envio entre las features, lo separamos en 3 features independientes.

In [5]:
#df = pd.get_dummies(df, columns=['shipment_type'])

Definimos las columnas que usaremos como features para el modelo, y cual sera la columna target.

In [6]:
#features = ['sender_zipcode', 'receiver_zipcode', 'service', 'shipment_type_express', 'shipment_type_standard', 'shipment_type_super']
features = ['sender_zipcode', 'receiver_zipcode', 'service']
target = 'target'

Antes de aplicar cualquier modelo de prediccion, graficamos la distribucion de los datos segun las diferentes features.

In [7]:
#sns.pairplot(
#    data=df,
#    vars=features,
#    hue=target)

## Regresion Logistica

Como modelo base, utilizaremos un pipeline de sklearn, que nos permita normalizar los datos y luego pasarlos a una Regresion Logistica Multiclase.

In [9]:
def drop_last_zip_digit(data):
    for row in data:
        row[0] = round(row[0] / 10)
        row[1] = round(row[1] / 10)
    return data

zip_features = [0, 1]
function_transformer = FunctionTransformer(drop_last_zip_digit, validate=False)

model = Pipeline([
    ('zip_cutter', ColumnTransformer(transformers=[('log', function_transformer, zip_features)])),
    ('normalizer', MinMaxScaler()),
    ('reduce_dim', PCA()),
    ('classifier', LogisticRegression(solver='lbfgs', 
                                      multi_class='multinomial',
                                      max_iter=500))
])

Definimos una fecha de corte para separar los datos de entrenamiento y test. Para asegurar que los datos no sean modificados, trabajamos sobre una copia del dataset original

In [10]:
copy = df.copy()

cut_off = '2019-03-20'
df_train = copy.query(f'date_visit <= "{cut_off}"')
df_test = copy.query(f'date_created > "{cut_off}"')

X_train = df_train[features].values.astype(np.float)
y_train = df_train[target].values

X_test = df_test[features].values.astype(np.float)
y_test = df_test[target].values

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((673645, 3), (673645,), (76378, 3), (76378,))

In [11]:
y_train = df_train[target].values
y_test = df_test[target].values
y_train.shape, y_test.shape

((673645,), (76378,))

Entrenamos nuestro modelo con los datos de entrenamiento y test definidos.

In [12]:
%%time
result = model.fit(X_train, y_train)
result

Wall time: 1min 2s


Pipeline(memory=None,
         steps=[('zip_cutter',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('log',
                                                  FunctionTransformer(accept_sparse=False,
                                                                      check_inverse=True,
                                                                      func=<function drop_last_zip_digit at 0x000002681CD4C048>,
                                                                      inv_kw_args=None,
                                                                      inverse_func=None,
                                                                      kw_args=None,
                                                                      pass_y='deprecated',
                                            

Tomamos los scores del resultado para intentar definir que tan acertado fue el modelo.

In [13]:
y_pred = model.predict(X_test)

metrics = {
    'accuracy': accuracy_score(y_test, y_pred),
    'precision': precision_score(y_test, y_pred, average='macro'),
    'recall': recall_score(y_test, y_pred, average='macro'),
    'f1_score': f1_score(y_test, y_pred, average='macro'),
}

metrics

C:\Users\femancus\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\femancus\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\femancus\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\femancus\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'tru

{'accuracy': 0.42414045929456123,
 'precision': 0.09267482806430831,
 'recall': 0.08913194095878259,
 'f1_score': 0.07790023656234467}

El modelo no se comporta adecuadamente con los datos disponibles. 

## Arboles de Desicion

Iniciamos con una copia del dataset original

In [10]:
copy = df.copy()

cut_off = '2019-03-20'
df_train = copy.query(f'date_visit <= "{cut_off}"')
df_test = copy.query(f'date_created > "{cut_off}"')

X_train = df_train[features].values.astype(np.float)
y_train = df_train[target].values

X_test = df_test[features].values.astype(np.float)
y_test = df_test[target].values

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((673645, 3), (673645,), (76378, 3), (76378,))

Creamons un pileline de skLearn

In [14]:
model = Pipeline([
    ('zip_cutter', ColumnTransformer(transformers=[('log', function_transformer, zip_features)])),
    ('normalizer', MinMaxScaler()),
    ('reduce_dim', PCA()),
    ('classifier', XGBoostClassifier())
])

NameError: name 'XGBoostClassifier' is not defined

## KNN

Iniciamos con una copia del dataset original

In [10]:
copy = df.copy()

cut_off = '2019-03-20'
df_train = copy.query(f'date_visit <= "{cut_off}"')
df_test = copy.query(f'date_created > "{cut_off}"')

X_train = df_train[features].values.astype(np.float)
y_train = df_train[target].values

X_test = df_test[features].values.astype(np.float)
y_test = df_test[target].values

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((673645, 3), (673645,), (76378, 3), (76378,))

Creamons un pileline para estimar Kvecinos cercanos

## Regresion

Iniciamos con una copia del dataset original

In [10]:
copy = df.copy()

cut_off = '2019-03-20'
df_train = copy.query(f'date_visit <= "{cut_off}"')
df_test = copy.query(f'date_created > "{cut_off}"')

X_train = df_train[features].values.astype(np.float)
y_train = df_train[target].values

X_test = df_test[features].values.astype(np.float)
y_test = df_test[target].values

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((673645, 3), (673645,), (76378, 3), (76378,))

## Prediccion

Iniciamos con una copia del dataset original

In [10]:
copy = df.copy()

cut_off = '2019-03-20'
df_train = copy.query(f'date_visit <= "{cut_off}"')
df_test = copy.query(f'date_created > "{cut_off}"')

X_train = df_train[features].values.astype(np.float)
y_train = df_train[target].values

X_test = df_test[features].values.astype(np.float)
y_test = df_test[target].values

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((673645, 3), (673645,), (76378, 3), (76378,))